In [ ]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Mindsay/mathieu/Benter-Project

In [2]:
from models.horse import Horse
from database.setup import create_sqlalchemy_session

from utils.music import parse_music


In [10]:
%%time

with create_sqlalchemy_session() as db_session:
    horses=[db_session.query(Horse).order_by(Horse.id.asc()).first()]
    for horse in horses:
        for runner in horse.runners:
            print(runner.date, runner.sex, runner.age,  runner.race.horse_show.race_track.country_name, runner.music)
        break

2005-06-18 H 7 FRANCE 6o 1o (04) 4o 2o 1o 1o 3o
2005-09-28 H 7 FRANCE 5o 6o 1o (04) 4o 2o 1o 1o
2005-10-22 H 7 FRANCE 6o 5o 6o 1o (04) 4o 2o 1o
2005-11-13 H 7 FRANCE 3o 6o 5o 6o 1o (04) 4o 2o
2007-09-29 H 7 FRANCE (05) 3o 3o 6o 5o 6o 1o (04) 4o 2o 1o 1o 3o 3o
2007-11-04 H 7 FRANCE 2o (05) 3o 3o 6o 5o 6o 1o (04) 4o 2o 1o 1o 3o
2008-03-19 H 7 FRANCE (07) 8o 2o (05) 3o 3o 6o 5o 6o 1o (04) 4o 2o 1o 1o
2008-04-01 H 7 FRANCE 4o (07) 8o 2o (05) 3o 3o 6o 5o 6o 1o (04) 4o 2o 1o
CPU times: user 18 ms, sys: 100 µs, total: 18.1 ms
Wall time: 18.9 ms


In [13]:
parse_music(runner.music, verbose=True)

Found events: []


ParsedMusic(win_rate=nan, mean_place=nan, n_races_in_music=0)

In [14]:
runner.music

'4o (07) 8o 2o (05) 3o 3o 6o 5o 6o 1o (04) 4o 2o 1o'

In [15]:
runner.race.type

'Haies'

In [16]:
import re

In [ ]:
s=

In [39]:
from collections import namedtuple
from typing import List

MusicEvent = namedtuple("MusicEvent", ["rank", "race_type", "year"])



current_year=2008

music_events:List[MusicEvent]=[]
events=re.findall(r"(([ATRDN]|\d)([a-z]))|(\((\d{1,2})\))", runner.music)
for event in events:
    race_event, rank, race_type, _, year = event
    if year:
        assert not race_event
        next_current_year = int(f'20{year}')
        # We read music from left to right to go backward in time
        assert next_current_year < current_year
        current_year=next_current_year
        continue
    assert not year
    music_events.append(MusicEvent(rank=rank, race_type=race_type, year=current_year))
    

In [40]:
music_events

[MusicEvent(rank='4', race_type='o', year=2008),
 MusicEvent(rank='8', race_type='o', year=2007),
 MusicEvent(rank='2', race_type='o', year=2007),
 MusicEvent(rank='3', race_type='o', year=2005),
 MusicEvent(rank='3', race_type='o', year=2005),
 MusicEvent(rank='6', race_type='o', year=2005),
 MusicEvent(rank='5', race_type='o', year=2005),
 MusicEvent(rank='6', race_type='o', year=2005),
 MusicEvent(rank='1', race_type='o', year=2005),
 MusicEvent(rank='4', race_type='o', year=2004),
 MusicEvent(rank='2', race_type='o', year=2004),
 MusicEvent(rank='1', race_type='o', year=2004)]

In [36]:
runner.music

'4o (07) 8o 2o (05) 3o 3o 6o 5o 6o 1o (04) 4o 2o 1o'

In [26]:
race_type

'o'

In [27]:
_

''

In [28]:
year

''

In [29]:
race_type

'o'